In [2]:
import sys
import os
import math

In [3]:
import pysat
pysat.__version__

'0.1.7.dev12'

In [4]:
from dpll import DPLL
from utils import *

In [5]:
test_sats_path = 'test sudokus'

In [6]:
input_mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9, 'A': 10, 'B': 11, "C": 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16}

inverse_input_mapping = {v:k for k, v in input_mapping.items()}

# Experiment

In [7]:
from utils import exit_after

In [8]:
import random
import pandas as pd
from utils import CNFsetup2matrix, matprint, print_solved

In [9]:
import time
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [10]:
def run_experiment(rules_path, setups_path, 
                   variable_selection_method='JWTS',
                   sudoku_num=50,
                   duration_limit_sec=60,
                   verbose=0):
    rules = read_DIMACS(rules_path)
    
    with open(setups_path) as f:
        sudoku_list = f.readlines()
    random.seed(42)
    sudoku_sample_indices = random.sample(range(0, len(sudoku_list)), sudoku_num)
    
    results_df = pd.DataFrame()

    for sudoku_index in sudoku_sample_indices:
        sudoku = sudoku_list[sudoku_index]
        
        sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku, inverse_input_mapping)
        
        givens, givens_num, row_fullness, col_fullness = get_setup_info(sudoku, sudoku_setup_CNF, sudoku_size)
        if verbose:
            matprint(CNFsetup2matrix(sudoku_setup_CNF, sudoku_size))
        
        clauses = rules + sudoku_setup_CNF
        solver = DPLL(clauses, 
                      variable_selection_method=variable_selection_method, 
                      verbose=verbose,
                      sudoku_size=sudoku_size, 
                      row_fullness=row_fullness, 
                      col_fullness=col_fullness)
                
        time_limit = False
        start = time.time()
        try:
            solution = exit_after(duration_limit_sec)(solver.backtrack)(solver.clauses, partial_assignment=[], split_literal=tuple())     
        except:
            print(f'Was not solved in {duration_limit_sec} seconds. Skip')
            solution = False, None
            time_limit = True
        solution_time = time.time() - start     
        
        if verbose and solution[0]:
            print_solved(solution[1], sudoku_size)

        results_df.loc[len(results_df), ['rules_path', 'setups_path', 'setup_index', 'givens', 'givens_num', 'row_fullness', 'col_fullness', 'variable_selection_method', 'sat', 'solution', 'time_limit', 'time', 'backtracks']] = rules_path, setups_path, sudoku_index, givens, givens_num, row_fullness, col_fullness, variable_selection_method, *solution, time_limit, solution_time, solver.backtrack_counter
        display(results_df.tail(1))
        
    return results_df

In [ ]:
variable_selection_method = 'JWTS'
sudoku_num = 50
verbose = 0

results_df = run_experiment(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'),
                            os.path.join(test_sats_path, '1000 sudokus.txt'), # 1000 sudokus, 4x4
                            variable_selection_method=variable_selection_method,
                            sudoku_num=sudoku_num,
                            duration_limit_sec=60,
                            verbose=verbose,
              )
results_df.to_csv(f"{sudoku_num}_launches_size_9_{variable_selection_method}.csv", index=False)

,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
0,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,654.0,"{'.': 56, '4': 4, '8': 2, '6': 3, '2': 4, '5':...",25.0,"{1: 2, 2: 2, 3: 3, 4: 2, 5: 4, 6: 3, 7: 3, 8: ...","{1: 1, 2: 4, 3: 4, 4: 2, 5: 4, 6: 2, 7: 0, 8: ...",JWTS,True,"[(154, True), (151, False), (152, False), (153...",False,32.878074,1.0


,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
1,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,114.0,"{'.': 58, '8': 4, '3': 2, '7': 2, '9': 2, '1':...",23.0,"{1: 0, 2: 3, 3: 4, 4: 1, 5: 3, 6: 4, 7: 2, 8: ...","{1: 1, 2: 3, 3: 4, 4: 2, 5: 2, 6: 3, 7: 4, 8: ...",JWTS,True,"[(248, True), (241, False), (242, False), (243...",False,55.03604,215.0


Was not solved in 60 seconds. Skip


,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
2,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,25.0,"{'.': 58, '4': 3, '1': 4, '9': 3, '6': 1, '3':...",23.0,"{1: 1, 2: 4, 3: 2, 4: 1, 5: 4, 6: 1, 7: 3, 8: ...","{1: 2, 2: 4, 3: 2, 4: 3, 5: 2, 6: 3, 7: 4, 8: ...",JWTS,False,None,True,60.04439,339.0


Was not solved in 60 seconds. Skip


...

,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
10,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,692.0,"{'.': 56, '2': 3, '6': 4, '5': 4, '1': 2, '8':...",25.0,"{1: 0, 2: 3, 3: 6, 4: 0, 5: 4, 6: 4, 7: 1, 8: ...","{1: 4, 2: 2, 3: 1, 4: 3, 5: 2, 6: 6, 7: 2, 8: ...",JWTS,True,"[(262, True), (261, False), (263, False), (264...",False,35.728453,35.0


,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
11,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,758.0,"{'.': 55, '8': 4, '1': 4, '2': 3, '9': 4, '5':...",26.0,"{1: 3, 2: 3, 3: 4, 4: 4, 5: 4, 6: 0, 7: 2, 8: ...","{1: 2, 2: 3, 3: 4, 4: 3, 5: 3, 6: 3, 7: 2, 8: ...",JWTS,True,"[(138, True), (131, False), (132, False), (133...",False,55.262199,319.0


Was not solved in 60 seconds. Skip


,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
12,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,913.0,"{'3': 5, '4': 2, '8': 5, '.': 55, '2': 3, '6':...",26.0,"{1: 4, 2: 3, 3: 2, 4: 2, 5: 5, 6: 2, 7: 3, 8: ...","{1: 4, 2: 4, 3: 3, 4: 1, 5: 3, 6: 2, 7: 3, 8: ...",JWTS,False,None,True,60.072356,327.0


,rules_path,setups_path,setup_index,givens,givens_num,row_fullness,col_fullness,variable_selection_method,sat,solution,time_limit,time,backtracks
13,test sudokus\sudoku-rules-9x9.txt,test sudokus\1000 sudokus.txt,558.0,"{'.': 56, '4': 2, '5': 4, '3': 4, '2': 3, '7':...",25.0,"{1: 3, 2: 3, 3: 3, 4: 3, 5: 1, 6: 3, 7: 3, 8: ...","{1: 2, 2: 3, 3: 2, 4: 5, 5: 3, 6: 2, 7: 4, 8: ...",JWTS,True,"[(164, True), (161, False), (162, False), (163...",False,32.609235,1.0


In [ ]:
results_df.to_csv(f"{sudoku_num}_launches_size_9_{variable_selection_method}.csv", index=False)

This is a two-sided test for the null hypothesis that 2 related or repeated samples have identical average (expected) values. 

https://en.wikipedia.org/wiki/T-test#Dependent_t-test_for_paired_samples

In [ ]:
from scipy.stats import ttest_rel
df1 = pd.read_csv("50_launches_size_9_fullnes.csv")
df2 = pd.read_csv("50_launches_size_9_random.csv")

df1 = df1[df1['sat']]
df2 = df2[df2['sat']]

df1 = df1[df1['setup_index'].isin(df2['setup_index'])]
df2 = df2[df2['setup_index'].isin(df1['setup_index'])]

assert df1.shape == df2.shape, 'Dataframes should have the same lengths'

stat, p = ttest_rel(df1['time'], df2['time'])
print(f'statistic {stat}, p {p}')
p_sig = 0.05
if p > p_sig:
    print("Considering a significance level = 0.05, Paired Student's t-test fails to reject the null hypothesis (p-value is greater than significance level). Therefore, we can claim that time variances are homogeneous/equal for both approaches")
else:
    print("Considering a significance level = 0.05, Paired Student's t-test rejects the null hypothesis (p-value is less or equal than significance level). Therefore, we can claim that time variances are not homogeneous/equal for both approaches")    

In [ ]:
import matplotlib.pyplot as plt
bins =[30,35,40,45,50,55,60]
plt.hist([df1['time'], df2['time']], bins, label=['fullnes', 'random'])
plt.title("Time histograms", fontsize=16)
plt.legend();

In [ ]:
## Experiment takes number of runs. In each run a number of sudoku's is solved. The average backtracks and time of solving the sudoku's 
## is stored. Eventually, the average for every run is stored in d. 

# To change method, adjust code above
def experiment_runs(sudoku_num, run_num):
    avg_time = []
    avg_bt = []
    ex_run = []
    run = 1
    for i in range (0,run_num):
        results_df = run_experiment(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'),
        os.path.join(test_sats_path, '1000 sudokus.txt'), # 1000 sudokus, 4x4
        sudoku_num)
        avg_time.append(np.average(results_df.time))
        avg_bt.append(np.average(results_df.backtracks))
        ex_run.append(run)
        run += 1
    dict = {'Run with ' + str(sudoku_num) + 'setups': ex_run , 'Time': avg_time, 'Backtracks': avg_bt}
    d = pd.DataFrame(dict)

experiment_runs(10,1)

# experiment 2

In [ ]:
def entropy(string):
    #Calculates the Shannon entropy of a string

    # get probability of chars in string
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

    return entropy


In [ ]:
## Calculate the distribution of amount of givens in the test-sudoku's
with open(os.path.join(test_sats_path, '1000 sudokus.txt')) as f:
    amount_given = []
    lines = f.readlines()
    for line in lines:
        dots = line.count('.')
        given = 81 - dots
        amount_given.append(given)
    k =  dict((x, amount_given.count(x)) for x in set(amount_given))
    print(k)
    

In [ ]:
def getsudokusbygiven():
    with open(os.path.join(test_sats_path, '1000 sudokus.txt')) as f:
        sudokus = {}
        lines = f.readlines()
        for line in lines:
            dots = line.count('.')
            given = 81 - dots
            if given in sudokus:
                sudokus[given].append(lines.index(line))
            else:
                sudokus[given]=[lines.index(line)]
        return sudokus

In [ ]:
def run_experiment2(rules_path,setups_path,setups, 
                   variable_selection_method='JWTS',
                   sudoku_num=50,
                   duration_limit_sec=300,
                   verbose=0):
    rules = read_DIMACS(rules_path)
    with open(setups_path) as f:
        sudoku_list = f.readlines()
    sudoku_sample_indices = random.sample(range(0, len(setups)), sudoku_num)
    results_df = pd.DataFrame()
    for sudoku_setup in setups[:sudoku_num]:
        sudoku=sudoku_list[sudoku_setup]
        sudoku_entropy=entropy(sudoku)
        setup_index = sudoku_setup
        sudoku_setup_CNF, sudoku_setup_DIMACS, sudoku_size = line2CNF(sudoku, inverse_input_mapping)
        
        givens, givens_num, row_fullness, col_fullness = get_setup_info(sudoku, sudoku_setup_CNF, sudoku_size)
        if verbose:
            matprint(CNFsetup2matrix(sudoku_setup_CNF, sudoku_size))
        
        clauses = rules + sudoku_setup_CNF
        solver = DPLL(clauses, 
                      variable_selection_method=variable_selection_method, 
                      verbose=verbose,
                      sudoku_size=sudoku_size, 
                      row_fullness=row_fullness, 
                      col_fullness=col_fullness)
                
        time_limit = False
        start = time.time()
        try:
            solution = exit_after(duration_limit_sec)(solver.backtrack)(solver.clauses, partial_assignment=[], split_literal=tuple())     
        except:
            print(f'Was not solved in {duration_limit_sec} seconds. Skip')
            solution = False, None
            time_limit = True
        solution_time = time.time() - start     
        
        if verbose and solution[0]:
            print_solved(solution[1], sudoku_size)

        results_df.loc[len(results_df), ['rules_path', 'setup_index', 'givens', 'givens_num', 'row_fullness', 'col_fullness', 'variable_selection_method', 'sat', 'solution', 'time_limit', 'time', 'backtracks','entropy']] = rules_path,  setup_index, givens, givens_num, row_fullness, col_fullness, variable_selection_method, *solution, time_limit, solution_time, solver.backtrack_counter,sudoku_entropy
        display(results_df.tail(1))
        
    return results_df

In [ ]:
def experiment2_runs(givens,sudokucount):
    avg_time = []
    avg_bt = []
    ex_run = []
    sudokus = getsudokusbygiven()
    sudokunum=sudokucount
    for i in givens:
        results_df = run_experiment2(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'),
                             os.path.join(test_sats_path, '1000 sudokus.txt'),
                        sudokus[i] , # 1000 sudokus, 4x4
                        variable_selection_method='random',
                        sudoku_num=sudokunum,
                        duration_limit_sec=300,
                        verbose=0)
        avg_time.append(np.average(results_df.time))
        avg_bt.append(np.average(results_df.backtracks))
        ex_run.append(i)
    dict = {'Run with ' + str(sudoku_num) + 'setups': ex_run , 'Average time': avg_time, 'Average backtracks': avg_bt}
    d = pd.DataFrame(dict)
    return d



In [ ]:
givens = [22,23,24,25,26,27,28]
sudokucount = 10
experiment2_runs(givens,sudokucount)

# experiment 2.2

In [ ]:
sudokus = getsudokusbygiven()
df = results_df = run_experiment2(os.path.join(test_sats_path, 'sudoku-rules-9x9.txt'),
                             os.path.join(test_sats_path, '1000 sudokus.txt'),
                        sudokus[25] , # 1000 sudokus, 4x4
                        variable_selection_method='random',
                        sudoku_num=50,
                        duration_limit_sec=300,
                        verbose=0)
df.sort_values(by=['entropy'])
display(df)

linear regression

In [ ]:
from scipy.stats import ttest_rel,
df1 = pd.read_csv("50_launches_size_9_JWTS.csv")
df2 = pd.read_csv("50_launches_size_9_random.csv")

df1 = df1[df1['sat']]
df2 = df2[df2['sat']]

df1 = df1[df1['setup_index'].isin(df2['setup_index'])]
df2 = df2[df2['setup_index'].isin(df1['setup_index'])]

assert df1.shape == df2.shape, 'Dataframes should have the same lengths'

stat, p = linregress(df1['time'], df2['time'])
print(f'statistic {stat}, p {p}')
p_sig = 0.05
if p > p_sig:
    print("Considering a significance level = 0.05, Paired Student's t-test fails to reject the null hypothesis (p-value is greater than significance level). Therefore, we can claim that time variances are homogeneous/equal for both approaches")
else:
    print("Considering a significance level = 0.05, Paired Student's t-test rejects the null hypothesis (p-value is less or equal than significance level). Therefore, we can claim that time variances are not homogeneous/equal for both approaches")    

In [ ]:
import matplotlib.pyplot as plt
ax1 = results_df.plot.scatter(x='backtracks',
                      y='time',
                      c='DarkBlue')